# Continuous Tomography

This Juypter Notebook is a tutorial for generating simulated data for the publication: ..................

Table of Contents
[Setup](#21-manage-imports)
[Tests](#tests)


### 2.1. Manage Imports 
This section is to manage imports for libraries necessary to run the code. Data will be simulated from the Nanocage 3D volume.

In [ ]:
import astra
print(astra.use_cuda())

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes
import pandas as pd

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()
pd.set_option('display.max_rows', None)
import astra
print(astra.use_cuda())

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes.alignments
import tomobase.tiltschemes
import mrcfile


vol =  tomobase.phantoms.nanocage()
scheme = tomobase.tiltschemes.Binary(-64,64, 8)
angles = np.array([scheme.get_angle() for i in range(65)])
sino = tomobase.processes.project(vol, angles)
tomobase.processes.alignments.poisson_noise(sino, 10)
sino.data = np.transpose(sino.data,(2,1,0))
sino.to_file('sino.mrc')

#meta={'angles': self.angles, 'times': self.times}

In [ ]:
import tomobase
import copy
from skimage.metrics import peak_signal_noise_ratio, structural_similarity


vol =  tomobase.phantoms.nanocage()
schemes = get_required_schemes()

for scheme in schemes:
    angles = np.array([scheme.get_angle() for i in range(65)])
    sino = tomobase.processes.project(vol, angles)
    rec = tomobase.processes.reconstruct_weighted_sirt(sino, 100, weighted=True)
    rmse = np.sqrt(np.mean((vol.data - rec.data)**2))
    ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
    print('Incremental', rmse, ssim)


rec.to_file('inc.rec')
print('Done')
sinos = [sino_inc, sino_bin, sino_grs]
for sino in sinos:
    noises = [0, 0.1, 1.0, 10.0]
    for noise in noises:
        if noise == 0: 
            sino_noisy = sino
        else:
            sino_noisy =  tomobase.processes.alignments.poisson_noise(sino_noisy, noise, inplace=False)
        rec = tomobase.processes.reconstruct_weighted_sirt(sino_noisy, 100, True, True)
        vol2 = copy.deepcopy(vol)
        if noise > 0:
            vol2 = vol2.data*noise
        rmse = np.sqrt(np.mean((vol.data - rec.data)**2))
        ssim = structural_similarity(vol.data, rec.data, data_range=rec.data.max() - rec.data.min())
        print(noise, rmse, ssim)




### 2.2 TiltScheme Setup
The following section setups a loop to acquire all base tilt scheme angles required in this study

In [ ]:

# return all tiltschemes for basic tests
def generate_schemes(scheme_selection=None):
    schemes = []
    labels = []
    angles = []
    
    if scheme_selection is None:
        scheme_selection = [0,1,2]
    
    for set in scheme_selection: 
        match set:
            case 0:
                incremental_schemes = [tomobase.tiltschemes.Incremental(-64, 64, 2), 
                                        tomobase.tiltschemes.Incremental(-64, 64, 4), 
                                        tomobase.tiltschemes.Incremental(-64, 64, 8), 
                                        tomobase.tiltschemes.Incremental(-64, 64, 16)]
                incremental_angles = [65, 33, 17, 9]
                incremental_labels = ['Incremental', 'Incremental', 'Incremental', 'Incremental']
                schemes.extend(incremental_schemes)
                angles.extend(incremental_angles)
                labels.extend(incremental_labels)
            case 1:
                binary_schemes = [tomobase.tiltschemes.Binary(-64, 64,k=2), 
                                    tomobase.tiltschemes.Binary(-64, 64,k=2), 
                                    tomobase.tiltschemes.Binary(-64, 64,k=2),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8), 
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=2, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=4, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False),
                                    tomobase.tiltschemes.Binary(-64, 64,k=8, isbidirectional=False)]
                binary_angles = [65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9, 65, 33, 17, 9]
                binary_labels = ['Binary 2b', 'Binary 2b', 'Binary 2b', 'Binary 2b', 'Binary 4b', 'Binary 4b', 'Binary 4b', 'Binary 4b', 'Binary 8b', 'Binary 8b', 'Binary 8b', 'Binary 8b', 'Binary 2u', 'Binary 2u', 'Binary 2u', 'Binary 2u', 'Binary 4u', 'Binary 4u', 'Binary 4u', 'Binary 4u', 'Binary 8u', 'Binary 8u', 'Binary 8u', 'Binary 8u']
                
                schemes.extend(binary_schemes)
                angles.extend(binary_angles)
                labels.extend(binary_labels)
            case 2:
                grs_schemes =  [tomobase.tiltschemes.GRS(-64, 64,0),
                                tomobase.tiltschemes.GRS(-64, 64,0),
                                tomobase.tiltschemes.GRS(-64, 64,0),
                                tomobase.tiltschemes.GRS(-64, 64,0)]
                grs_angles = [65, 33, 17, 9]
                grs_labels = ['GRS', 'GRS', 'GRS', 'GRS']
                schemes.extend(grs_schemes)
                angles.extend(grs_angles)
                labels.extend(grs_labels)
    return schemes, angles, labels


In [ ]:
def get_required_schemes():
    schemes = [tomobase.tiltschemes.Incremental(-64, 64, 2), 
               tomobase.tiltschemes.Binary(-64, 64,k=8), 
               tomobase.tiltschemes.GRS(-64, 64,0)]
    return schemes

## Tests
The following section outlines the tests performed in this study

Table of Contents

[3.1. Display Angles](#31-display-angles) - Displays the angles acquired for novel schemes

[3.2. Backlash Corrections](#32-backlash) - Displays Backlash amount and correction

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes
import pandas as pd

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()
pd.set_option('display.max_rows', None)
import astra
print(astra.use_cuda())

def get_required_schemes():
    schemes = [ tomobase.tiltschemes.Incremental(-64, 64, 2),
                tomobase.tiltschemes.Binary(-64, 64,k=8), 
               tomobase.tiltschemes.GRS(-64, 64,0)]
    return schemes

schemes = get_required_schemes()
df = pd.DataFrame(columns=['Tilt Scheme', 'Angles','Noise', 'MAE', 'SSIM'])
for scheme in schemes:
    for noise in [0]:
        angles = np.array([scheme.get_angle() for i in range(65)])
        sinogram = tomobase.processes.project(vol, angles)
        print(len(sinogram.angles))
        if noise == 0:
            sinogram_noisy = sinogram
        else:
            sinogram_noisy = tomobase.processes.alignments.add_noise(sinogram)
        rec = tomobase.processes.reconstruct_weighted_sirt(sinogram_noisy, iterations=100, weighted=False)
        #rec = tomobase.processes.reconstruct(sinogram_noisy, 'sirt', iterations=100)
        rmse = np.sqrt(np.mean((vol.data-rec.data)**2))
        ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
        print(rmse, ssim)
        new_row = {'Tilt Scheme': scheme, 'Angles': len(angles), 'Noise': noise, 'MAE': rmse, 'SSIM': ssim}
        new_row_df = pd.DataFrame([new_row])
    
        df = pd.concat([df, new_row_df], ignore_index=True)
    
display(df)

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes
import pandas as pd

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()
pd.set_option('display.max_rows', None)
import astra
print(astra.use_cuda())

def get_required_schemes():
    schemes = [ tomobase.tiltschemes.Incremental(-64, 64, 2),
                tomobase.tiltschemes.Binary(-64, 64,k=8), 
               tomobase.tiltschemes.GRS(-64, 64,0)]
    return schemes

schemes = get_required_schemes()
df = pd.DataFrame(columns=['Tilt Scheme', 'Angles','Noise', 'MAE', 'SSIM'])
for scheme in schemes:
    for noise in [0]:
        angles = np.array([scheme.get_angle() for i in range(65)])
        sinogram = tomobase.processes.project(vol, angles)
        print(len(sinogram.angles))
        if noise == 0:
            sinogram_noisy = sinogram
        else:
            sinogram_noisy = tomobase.processes.alignments.add_noise(sinogram)
        rec = tomobase.processes.reconstruct_weighted_sirt(sinogram_noisy, iterations=100, weighted=False)
        #rec = tomobase.processes.reconstruct(sinogram_noisy, 'sirt', iterations=100)
        rmse = np.sqrt(np.mean((vol.data-rec.data)**2))
        ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
        print(rmse, ssim)
        new_row = {'Tilt Scheme': scheme, 'Angles': len(angles), 'Noise': noise, 'MAE': rmse, 'SSIM': ssim}
        new_row_df = pd.DataFrame([new_row])
    
        df = pd.concat([df, new_row_df], ignore_index=True)
    
display(df)

### 3.1. Display Angles
The purpose of this test is just to be able to read off the angles for the binary tiltschemes. Two almost identical scheme sets are presented:

1. Unidirectional (u) -  once the highest angle in the set is reached the tiltseries is reverted back to the minimum angle with an offset.
2. Bidirectional (b) -  Once the highest angle in the set is reached the tiltseries is offset than angles are collected backwards.

Both schemes should have collected the exact same angles in a set of k+1 projections - just in a different order. 

In [ ]:

schemes, n_angles, labels = generate_schemes([1])
coloumns = []

for i in range(len(schemes)):
    if n_angles[i]==65:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])])
        coloumns.append(angles)

_dict = {
    'Binary 2b': coloumns[0],
    'Binary 4b': coloumns[1],
    'Binary 8b': coloumns[2],
    'Binary 2u': coloumns[3],
    'Binary 4u': coloumns[4],
    'Binary 8u': coloumns[5],
}
df = pd.DataFrame(_dict)

display(df)

In [ ]:
schemes, n_angles, labels = generate_schemes([0, 1, 2])
df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'SSIM', 'PSNR', 'MAE'])
for i, scheme in enumerate(schemes):
    angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
    wedge_end = np.max(angles)
    incremental = tomobase.tiltschemes.Incremental(-64, wedge_end, (wedge_end+64)/(n_angles[i]-1))
    angles_inc = np.array([incremental.get_angle() for j in range(n_angles[i])], dtype=np.float64)
    
    sino = tomobase.processes.project(vol, angles)
    rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)

    ssim  = structural_similarity(vol.data, rec.data, data_range=1.0)
    psnr = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
    mae = np.mean(np.abs(vol.data - rec.data))
    
    new_row = {
        'Tilt Scheme': labels[i],
        'Number of Angles': n_angles[i],
        'SSIM': ssim,
        'PSNR': psnr,
        'MAE': mae
    }
    
    new_row_df = pd.DataFrame([new_row])
    df = pd.concat([df, new_row_df], ignore_index=True)

display(df)

### 3.1. Uniformity

s

In [ ]:
import copy

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'SSIM', 'MAE'])
scheme = tomobase.tiltschemes.Incremental(-64, 64, 2)
angles = np.array([scheme.get_angle() for i in range(65)])

sino = tomobase.processes.project(vol, angles)
#sino = tomobase.processes.alignments.weight_by_angle(sino)
#rec = tomobase.processes.reconstruct_weighted_sirt(sino, method="sirt", iterations=100)
rec = tomobase.processes.reconstruct_weighted_sirt(sino, iterations=100)
ssim  = structural_similarity(vol.data, rec.data, data_range=1.0)
mae = np.mean(np.abs(vol.data - rec.data))
new_row = {
    'Tilt Scheme': 'GRS',
    'Number of Angles': 65,
    'SSIM': ssim,
    'MAE': mae
}

new_row_df = pd.DataFrame([new_row])
df = pd.concat([df, new_row_df], ignore_index=True)
display(df)

### 3.2. Backlash

There is an artifact in the microscope gonioometer for bidirectional acquisition schemes. This section outlines its effects on the different tiltschemes



In [ ]:
schemes, n_angles, labels = generate_schemes()
backlash_value = 0.1

banned = [0, 1, 2, 3, 4, 5, 6]

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Backlash Value', 'Cumulated Backlash', 'Correction', 'SSIM without error', 'PSNR without error', 'SSIM with error', 'PSNR with error', 'SSIM corrected', 'PSNR corrected'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        sino = tomobase.processes.project(vol, angles)
        indices = np.where(np.diff(sino.angles) < 0)[0] + 1
        
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        ssim_without_error = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr_without_error = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
        
        cumulated_backlash = backlash_value * len(indices)
        sino.angles[indices] -= backlash_value
        
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        ssim_with_error = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr_with_error = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
         
        sino, correction = tomobase.processes.alignments.backlash_correct(sino, extend_return=True)
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        
        ssim_corrected = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr_corrected = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)

        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Backlash Value': backlash_value,
            'Cumulated Backlash': cumulated_backlash,
            'Correction': correction,
            'SSIM without error': ssim_without_error,
            'PSNR without error': psnr_without_error,
            'SSIM with error': ssim_with_error,
            'PSNR with error': psnr_with_error,
            'SSIM corrected': ssim_corrected,
            'PSNR corrected': psnr_corrected
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

print(df) 

In [ ]:
schemes, n_angles, labels = generate_schemes()
backlash_value = 0.5

banned = [0, 1, 2, 3, 4]

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Backlash Value', 'Cumulated Backlash', 'Correction', 'MAE without error', 'MAE with error', 'MAE corrected'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        sino = tomobase.processes.project(vol, angles)
        indices = np.where(np.diff(sino.angles) < 0)[0] + 1
        
        rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
        mae_without_error = np.mean(np.abs(vol.data - rec.data))
        
        cumulated_backlash = backlash_value * len(indices)
        sino.angles[indices] -= backlash_value
        
        if len(indices) > 0:
            rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
            mae_with_error = np.mean(np.abs(vol.data - rec.data))
            
            sino, correction = tomobase.processes.alignments.backlash_correct(sino, extend_return=True)
            rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
            mae_corrected = np.mean(np.abs(vol.data - rec.data))
        else:
            mae_with_error = mae_without_error
            mae_corrected = mae_without_error
            correction = 0

        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Backlash Value': backlash_value,
            'Cumulated Backlash': cumulated_backlash,
            'Correction': correction,
            'MAE without error': mae_without_error,
            'MAE with error': mae_with_error,
            'MAE corrected': mae_corrected
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

display(df)

In [ ]:
schemes, n_angles, labels = generate_schemes()
backlash_value = 0.5

banned = []

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Backlash Value', 'Cumulated Backlash'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        sino = tomobase.processes.project(vol, angles)
        indices = np.where(np.diff(sino.angles) < 0)[0] + 1
        
        cumulated_backlash = backlash_value * len(indices)
        sino.angles[indices] -= backlash_value

        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Backlash Value': backlash_value,
            'Cumulated Backlash': cumulated_backlash,
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

display(df)

### Missing Wedge

In [ ]:
schemes, n_angles, labels = generate_schemes()

banned = []

df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Missing Wedge'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        missing_wedge = 180 - (np.max(angles) - np.min(angles))


        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Missing Wedge': missing_wedge,
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

display(df)

In [ ]:
schemes, n_angles, labels = generate_schemes([1,2])
df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'SSIM', 'PSNR', 'MAE'])
for i, scheme in enumerate(schemes):
    angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
    wedge_end = np.max(angles)
    incremental = tomobase.tiltschemes.Incremental(-64, wedge_end, (wedge_end+64)/(n_angles[i]-1))
    angles_inc = np.array([incremental.get_angle() for j in range(n_angles[i])], dtype=np.float64)
    
    sino = tomobase.processes.project(vol, angles_inc)
    rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)

    ssim  = structural_similarity(vol.data, rec.data, data_range=1.0)
    psnr = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
    mae = np.mean(np.abs(vol.data - rec.data))
    
    new_row = {
        'Tilt Scheme': labels[i],
        'Number of Angles': n_angles[i],
        'SSIM': ssim,
        'PSNR': psnr,
        'MAE': mae
    }
    
    new_row_df = pd.DataFrame([new_row])
    df = pd.concat([df, new_row_df], ignore_index=True)

display(df)


In [ ]:
display(df)

### 3.3 Translational Alignment

In [ ]:
schemes, n_angles, labels = generate_schemes()
banned = []
df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Alignment Error', 'Measurement Error'])
for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        experiment_offset =[]
        for j in range(10):
            ts = tomobase.processes.project(vol, angles)
            ts = tomobase.processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)

            sorted_indices = np.argsort(ts.angles)
            ts.angles = ts.angles[sorted_indices]
            ts.data = ts.data[:, :, sorted_indices]
            ts, shifts = tomobase.processes.alignments.translational_misalignment(ts, 0.2, 0.2, extend_return=True)
            ts, shifts_corrected = tomobase.processes.alignments.align_sinogram_xcorr(ts, extend_return=True)   
        
            offsets = (shifts_corrected + shifts)%1024
            offsets[offsets>512] = np.abs(offsets[offsets>512] - 1024)
            experiment_offset.append(np.mean(offsets))
        
        offsets = np.mean(experiment_offset)
        experiment_offset = np.array(experiment_offset)
        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Alignment Error': offsets,
            'Measurement Error': (np.max(experiment_offset) - np.min(experiment_offset))/2
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

display(df)


In [ ]:
schemes, n_angles, labels = generate_schemes()
banned = []
df = pd.DataFrame(columns=['Tilt Scheme', 'Number of Angles', 'Alignment Error', 'Measurement Error', 'SSIM', 'SSIM Error','PSNR','PSNR Error', 'MAE', 'MAE Error'])
max_value = 0
min_value = 100000000

max_ssim = 0
min_ssim = 1000000000

max_psnr = 0
min_psnr = 1000000000

max_mae = 0
min_mae = 1000000000

for i, scheme in enumerate(schemes):
    if i not in banned:
        angles = np.array([schemes[i].get_angle() for j in range(n_angles[i])], dtype=np.float64)
        experiment_offset =[]
        ssims = []
        psnrs = []
        maes = []
        for j in range(10):
            ts = tomobase.processes.project(vol, angles)
            ts = tomobase.processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)

            sorted_indices = np.argsort(ts.angles)
            ts.angles = ts.angles[sorted_indices]
            ts.data = ts.data[:, :, sorted_indices]
            ts, shifts = tomobase.processes.alignments.translational_misalignment(ts, 0.2, 0.2, extend_return=True)
            ts, shifts_corrected = tomobase.processes.alignments.align_sinogram_xcorr(ts, extend_return=True)   
        
            offsets = (shifts_corrected + shifts)%1024
            offsets[offsets>512] = np.abs(offsets[offsets>512] - 1024)
            experiment_offset.append(np.mean(offsets))
            if np.max(offsets)>max_value:
                max_value = np.max(offsets)
            if np.min(offsets)<min_value:
                min_value = np.min(offsets)
            
            # Crop the sinogram to 307x307
            crop_height, crop_width = (307,307)
            height, width = ts.data.shape[:2]


            start_y = (height - crop_height) // 2
            end_y = start_y + crop_height
            start_x = (width - crop_width) // 2
            end_x = start_x + crop_width


            ts.data = ts.data[start_y:end_y, start_x:end_x, :]
            rec = tomobase.processes.reconstruct(ts, method="sirt", iterations=100)
            ssims.append(structural_similarity(vol.data, rec.data, data_range=1.0))
            psnrs.append(peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0))
            maes.append(np.mean(np.abs(vol.data - rec.data)))
                        
            if np.max(ssims)>max_ssim:
                max_ssim = np.max(ssims)
            if np.min(ssims)<min_ssim:
                min_ssim = np.min(ssims)
            if np.max(psnrs)>max_psnr:
                max_psnr = np.max(psnrs)
            if np.min(psnrs)<min_psnr:
                min_psnr = np.min(psnrs)
            if np.max(maes)>max_mae:
                max_mae = np.max(maes)
            if np.min(maes)<min_mae:
                min_mae = np.min(maes)
            
        offsets = np.mean(experiment_offset)
        experiment_offset = np.array(experiment_offset)
        new_row = {
            'Tilt Scheme': labels[i],
            'Number of Angles': n_angles[i],
            'Alignment Error': offsets,
            'Measurement Error': (max_value - min_value)/2,
            'SSIM': np.mean(ssims),
            'SSIM Error': (max_ssim - min_ssim)/2,
            'PSNR': np.mean(psnrs),
            'PSNR Error': (max_psnr - min_psnr)/2,
            'MAE': np.mean(maes),
            'MAE Error': (max_mae - min_mae)/2
        }

        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])

        # Concatenate the new row with the existing DataFrame
        df = pd.concat([df, new_row_df], ignore_index=True)

display(df)

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes
import pandas as pd

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()
pd.set_option('display.max_rows', None)
import astra
print(astra.use_cuda())

def get_required_schemes():
    schemes = [ tomobase.tiltschemes.Incremental(-64, 64, 2),
                tomobase.tiltschemes.Binary(-64, 64,k=8), 
               tomobase.tiltschemes.GRS(-64, 64,0)]
    return schemes

schemes = get_required_schemes()
for scheme in schemes:
    angles = np.array([scheme.get_angle() for i in range(65)])
    sinogram = tomobase.processes.project(vol, angles)
    sinogram_noisy = sinogram
    rec = tomobase.processes.reconstruct_weighted_sirt(sinogram_noisy, iterations=100, weighted=False)
    #rec = tomobase.processes.reconstruct(sinogram_noisy, 'sirt', iterations=100)
    rmse = np.sqrt(np.mean((vol.data-rec.data)**2))
    ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
    print(rmse, ssim)


In [2]:
import tomobase

tomobase.TOMOBASE_PROCESSES.update()
print("\n")
tomobase.TOMOBASE_PROCESSES.help()
print("\n")
tomobase.TOMOBASE_DATATYPES.help()
print("\n")
tomobase.TOMOBASE_TILTSCHEMES.help()
print("\n")
tomobase.TOMOBASE_TRANSFORM_CATEGORIES.help()
print("\n")

print("\n\n\n")
#tomobase.TOMOBASE_PROCESSES.help()


2025-03-30 17:40:38,854 - WARNING - Updating modlules <tomobase.registrations.processes.ProcessItemDict object at 0x0000021EA438DF10>
2025-03-30 17:40:38,854 - INFO - Module <tomobase.registrations.processes.ProcessItemDict object at 0x0000021EA438DF10>: tomobase processes
2025-03-30 17:40:38,854 - INFO - Found is_tomobase_process in project
2025-03-30 17:40:38,855 - INFO - Item to be added: Project
2025-03-30 17:40:38,855 - INFO - Checking category: DEFORM, DEFORM 0 2
2025-03-30 17:40:38,855 - INFO - Checking category: ALIGN, ALIGN 1 2
2025-03-30 17:40:38,856 - INFO - Checking category: PROJECT, PROJECT 2 2


Confirmed PROJECT 2 2


2025-03-30 17:40:38,858 - WARNING - Key PROJECT already exists in the dictionary
2025-03-30 17:40:38,858 - INFO - Checking category: RECONSTRUCT, RECONSTRUCT 3 2
2025-03-30 17:40:38,858 - INFO - Found is_tomobase_process in reconstruct
2025-03-30 17:40:38,858 - INFO - Item to be added: Reconstruct
2025-03-30 17:40:38,859 - INFO - Checking category: DEFORM, DEFORM 0 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,859 - INFO - Checking category: ALIGN, ALIGN 1 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,859 - INFO - Checking category: PROJECT, PROJECT 2 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,860 - INFO - Checking category: RECONSTRUCT, RECONSTRUCT 3 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,860 - INFO - Found is_tomobase_process in reconstruct_weighted_sirt
2025-03-30 17:40:38,860 - INFO - Item to be added: Reconstruct Weighted

Confirmed PROJECT 2 2


2025-03-30 17:40:38,865 - WARNING - Key PROJECT already exists in the dictionary
2025-03-30 17:40:38,866 - INFO - Checking category: RECONSTRUCT, RECONSTRUCT 3 2
2025-03-30 17:40:38,866 - INFO - Found is_tomobase_process in reconstruct
2025-03-30 17:40:38,866 - INFO - Item to be added: Reconstruct
2025-03-30 17:40:38,866 - INFO - Checking category: DEFORM, DEFORM 0 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,867 - INFO - Checking category: ALIGN, ALIGN 1 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,867 - INFO - Checking category: PROJECT, PROJECT 2 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,867 - INFO - Checking category: RECONSTRUCT, RECONSTRUCT 3 <tomobase.registrations.base.Item object at 0x0000021EA4802610>
2025-03-30 17:40:38,867 - INFO - Found is_tomobase_process in reconstruct_weighted_sirt
2025-03-30 17:40:38,867 - INFO - Item to be added: Reconstruct Weighted

...

2025-03-30 17:40:38,995 - WARNING - Key ALIGN already exists in the dictionary
2025-03-30 17:40:38,995 - INFO - Checking category: PROJECT, PROJECT 2 1
2025-03-30 17:40:38,995 - INFO - Checking category: RECONSTRUCT, RECONSTRUCT 3 1


2025-03-30 17:40:38,998 - INFO - Category: PROJECT
2025-03-30 17:40:38,998 - WARNING - Item PROJECT is not a dictionary!
2025-03-30 17:40:38,998 - INFO - Category: ALIGN
2025-03-30 17:40:38,999 - WARNING - Item ALIGN is not a dictionary!


2025-03-30 17:40:38,999 - INFO - IMAGE: IMAGE, 0
2025-03-30 17:40:39,000 - INFO - SINOGRAM: SINOGRAM, 1
2025-03-30 17:40:39,000 - INFO - VOLUME: VOLUME, 2


2025-03-30 17:40:39,001 - INFO - BINARY: BINARY, <class 'tomobase.tiltschemes.binary.Binary'>
2025-03-30 17:40:39,001 - INFO - GRS: GRS, <class 'tomobase.tiltschemes.grs.GRS'>
2025-03-30 17:40:39,001 - INFO - INCREMENTAL: INCREMENTAL, <class 'tomobase.tiltschemes.incremental.Incremental'>


2025-03-30 17:40:39,002 - INFO - DEFORM: DEFORM, 0
2025-03-30 17:40:39,002 - INFO - ALIGN: ALIGN, 1
2025-03-30 17:40:39,002 - INFO - PROJECT: PROJECT, 2
2025-03-30 17:40:39,003 - INFO - RECONSTRUCT: RECONSTRUCT, 3
